# Useful references

## Python + Numpy + Matplotlib + etc.

* Python Numpy Tutorial: http://cs231n.github.io/python-numpy-tutorial/
* Computational Statistics in Python: https://people.duke.edu/~ccc14/sta-663/
* Numpy for MATLAB users: https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html
* MATLAB synonymous commands in Python/NumPy: http://mathesaurus.sourceforge.net/

## NEURON (with Python)
* NEURON documentation: https://www.neuron.yale.edu/neuron/static/py_doc/index.html
* NEURON + Python tutorial: https://neuron.yale.edu/neuron/static/docs/neuronpython/index.html

## NMODL
* NEURON Extension to NMODL: https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/nmodl2.html
* NMODL: https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/nmodl.html#nmodl




# 1. Using custom mechanisms

Here we demonstrate how to compile custom active mechanisms. Most models of membrane/intracellular mechanisms are written in the NMODL language and the files should have an extension ".mod". In order to use those mod files (therefore active mechanisms), you first need to compile them, and, if necessary, direct NEURON to load the compiled library.

Here our first example is `MorrisLecar.mod`, which implements the Morris-Lecar ion channels, used in Rinzel and Ermentrout. If you look into the file, it looks like:

```
TITLE Morris-Lecar spiking dynamics

....

NEURON {
  SUFFIX ml
  USEION k READ ek WRITE ik
  USEION na READ ena WRITE ina
  NONSPECIFIC_CURRENT il
  RANGE gnabar, gkbar, gleak, el, ina, ik, il, w, winit
  RANGE phi, betam, gammam, betaw, gammaw
  THREADSAFE minf, winf, tauw
}

.....

```

The first part contains important information: `ml`, after `SUFFIX`, is the name of this mechanism in NEURON, just as `pas` and `hh` that we have seen before. `USEION` tells you which ions are used. In particular, it writes `ik` and `ina`, which correspond to the K+ and Na+ current. `RANGE` specifies the variables that you can read and write in NEURON.

To compile this, run `nrnivmodl` in the terminal or simply 

In [ ]:
%%bash

cd mod.files
nrnivmodl

Now let's create a single compartment cell,

In [ ]:
from neuron import h, gui
from neuron import load_mechanisms

import numpy as np
import matplotlib.pyplot as plt
load_mechanisms('./mod.files')

In [ ]:
soma = h.Section(name="soma")
soma.L = 20
soma.diam = 20

ic = h.IClamp(soma(0.5))

In [ ]:
soma.insert("hh2")
soma.psection()

In [ ]:
h.tstop = 300

ic.delay = 50
ic.dur = 200
ic.amp = 0.01

In [ ]:
h.celsius = 35

dt_rec = 0.1
t = h.Vector()
v = h.Vector()
v.record(soma(0.5)._ref_v, dt_rec)
t.record(h._ref_t, 0.1)

# nc = h.NetCon(soma(0.5)._ref_v, None, 0, 0, 1)
# tspike = h.Vector()
# nc.record(tspike)

h.run()

In [ ]:
_, ax = plt.subplots()

ax.plot(t, v)
ax.set(xlabel="time (ms)", ylabel = "v (mv)")

print(tspike.as_numpy())

## Pyramidal cell example

Please make sure that you reset the kernel at this point:

In [ ]:
from neuron import h, gui
from neuron import load_mechanisms
import libcell

import numpy as np
import matplotlib.pyplot as plt
load_mechanisms('./mod.files')


In [ ]:
cell = libcell.L23()

In [ ]:
libcell.init_active(cell, axon=True, soma=True, dend=True, dendNa=True, dendCa=True)

In [ ]:
cell.soma.psection()

In [ ]:
ic = h.IClamp(cell.soma(0.5))

In [ ]:
h.tstop = 300

ic.delay = 50
ic.dur = 200
ic.amp = 0.1


dt_rec = 0.1
t = h.Vector()
v = h.Vector()
v.record(cell.axon(1)._ref_v, dt_rec)
t.record(h._ref_t, 0.1)

h.v_init = -75
h.run()

In [ ]:
_, ax = plt.subplots()

ax.plot(t, v)
ax.set(xlabel="time (ms)", ylabel = "v (mv)")

## Adding synapses

Here we add synapses to the cell,

In [ ]:
def add_AMPAsyns(model, locs, stims, gmax=1, tau1=0.5, tau2=1):
    model.AMPAlist = []
    model.ncAMPAlist = []
    gmax = gmax/1000.   # Set in nS and convert to muS
    for loc, stim in zip(locs, stims):
        AMPA = h.Exp2Syn(float(loc[1]), sec=model.dends[int(loc[0])])
        AMPA.tau1 = tau1
        AMPA.tau2 = tau2
        NC = h.NetCon(stim, AMPA, 0, 0, gmax)
        model.AMPAlist.append(AMPA)
        model.ncAMPAlist.append(NC)

        
def add_NMDAsyns(model, locs, stims, gmax=1, tau1=2, tau2=20):
    model.NMDAlist = []
    model.ncNMDAlist = []
    gmax = gmax/1000.   # Set in nS and convert to muS
    for loc, stim in zip(locs, stims):
        NMDA = h.Exp2SynNMDA(float(loc[1]), sec=model.dends[int(loc[0])])
        NMDA.tau1 = tau1
        NMDA.tau2 = tau2
        NC = h.NetCon(stim, NMDA, 0, 0, gmax)
        x = float(loc[1])
        model.NMDAlist.append(NMDA)
        model.ncNMDAlist.append(NC)

        
def add_GABAsyns(model, locs, stims, gmax=0.5, tau1=0.1, tau2=4, rev=-80):
    model.GABAlist = []
    model.ncGABAlist = []
    gmax = gmax/1000.   # Set in nS and convert to muS
    for loc, stim in zip(locs, stims):
        GABA = h.Exp2Syn(float(loc[1]), sec=model.dends[int(loc[0])])
        GABA.tau1 = tau1
        GABA.tau2 = tau2
        GABA.e = rev
        NC = h.NetCon(stim, GABA, 0, 0, gmax)
        model.GABAlist.append(GABA)
        model.ncGABAlist.append(NC)

The first synapse will be inserted at cell.dend[0] and stimulated by an artificial spike generator.

In [ ]:
locs = [[0, 0.5]]

fexc = 10
exc_stims = []
for loc in locs:
    exc_stims.append(h.NetStimFD())
    exc_stims[-1].noise = 0 # maximally noisy stimulus
    exc_stims[-1].start = 150 # start from 0
    exc_stims[-1].duration = h.tstop
    exc_stims[-1].interval = 1000./fexc

add_AMPAsyns(cell, locs, exc_stims)
# add_NMDAsyns(cell, locs, exc_stims)


In [ ]:
h.tstop = 300

ic.delay = 50
ic.dur = 200
ic.amp = 0.0

dt_rec = 0.1
t = h.Vector()
v = h.Vector()
v.record(cell.soma(0.5)._ref_v, dt_rec)
t.record(h._ref_t, 0.1)

h.v_init = -75
h.run()

_, ax = plt.subplots()

ax.plot(t, v)
ax.set(xlabel="time (ms)", ylabel = "v (mv)")

Now let's try many more synapses, inserted at random locations!

In [ ]:
def genRandomLocs(model, nsyn):
    locs = []
    for s in np.arange(0,nsyn):
        dend = np.random.randint(low=0, high=len(model.dends))
        pos = np.random.uniform()
        locs.append([dend, pos])
    return locs

In [ ]:
# 100 excitatory and 20 inhibitory synapses
n_exc = 100
n_inh = 20

exc_locs = genRandomLocs(cell, n_exc)
inh_locs = genRandomLocs(cell, n_inh)
inh_locs

In [ ]:
fexc = 10
exc_stims = []
for loc in exc_locs:
    exc_stims.append(h.NetStimFD())
    exc_stims[-1].noise = 1 # maximally noisy stimulus
    exc_stims[-1].start = 0 # start from 0
    exc_stims[-1].duration = h.tstop
    exc_stims[-1].interval = 1000./fexc

add_AMPAsyns(cell, locs, exc_stims)
add_NMDAsyns(cell, locs, exc_stims)

finh = 40
inh_stims = []
for loc in inh_locs:
    inh_stims.append(h.NetStimFD())
    inh_stims[-1].noise = 1 # maximally noisy stimulus
    inh_stims[-1].start = 0 # start from 0
    inh_stims[-1].duration = h.tstop
    inh_stims[-1].interval = 1000./fexc

add_AMPAsyns(cell, exc_locs, exc_stims)
add_NMDAsyns(cell, exc_locs, exc_stims)
add_GABAsyns(cell, inh_locs, inh_stims)


# 2. Cell objects

**Make sure that you reset the kernel at this point.** 

In Python, you can define custom data types that you can use to organize data and related operations. For example, so far we have worked with cell models in NEURON, which have common attributes and operations as:

1. Sections and segments,
2. Combining sections to define the morphology of a cell,
3. Active mechanisms and inserting them,
4. Synapses and connecting them to other cells, etc.


We can collect these into a cell object as follows:

In [ ]:
from neuron import h, gui
from neuron import load_mechanisms

import numpy as np
import matplotlib.pyplot as plt
load_mechanisms('./mod.files')


class Cell:
    """ Cell class"""
    
    # When a cell object is created this function is called first.
    # Note "self" in definition
    def __init__(self):
        
        self.synlist = []      # list of synapses in this cell

        # here are some operations to create a cell
        self.create_sections()
        self.build_topology()
        self.build_subsets()
        self.define_geometry()
        self.define_biophysics()
        self.create_synapses()

    # At this stage, we leave most of them unimplemented
    # This technique is called "the Template Method pattern" (https://sourcemaking.com/design_patterns/template_method)
    def create_sections(self):
        raise NotImplementedError("create_section() is not implemented.")

    def build_topology(self):
        raise NotImplementedError("build_topology() is not implemented.")
    
    def build_subsets(self):
        raise NotImplementedError("build_subsets() is not implemented.")

    def define_geometry(self):
        raise NotImplementedError("define_geometry() is not implemented.")

    def define_biophysics(self):
        raise NotImplementedError("define_biophysics() is not implemented.")

    def create_synapses(self):
        raise NotImplementedError("create_synapses() is not implemented.")
    
    # Here are something that will be commonly used by every cell
    # built based on this template
    
    def connect2target(self, target, thresh=0):
        """Make a new NetCon with this cell's membrane
        potential at the soma as the source (i.e. the spike detector)
        onto the target passed in (i.e. a synapse on a cell).
        Subclasses may override with other spike detectors."""
        nc = h.NetCon(self.soma(1)._ref_v, target, sec = self.soma)
        nc.threshold = thresh
        return nc


In [ ]:
cell1 = Cell()

It will generate an error whenever you fail to provide any necessary step to define a proper cell. To implement those steps, we _subclass_ `Cell` as:

In [ ]:
class MorrisLecar(Cell):
    """Single compartment passive neuron"""
    
    def create_sections(self):
        """create a soma"""
        self.soma = h.Section(name="soma", cell=self)

    def build_topology(self):
        pass # single compartment
    
    def build_subsets(self):
        pass # single compartment
    
    def define_geometry(self):
        self.soma.L = 15
        self.soma.diam = 15

    def define_biophysics(self):
        self.soma.insert('ml')
        
    def create_synapses(self):
        self.synlist.append(h.Exp2Syn(self.soma(0.5))) # Excitatory
        self.synlist[-1].tau2 = 3.0
        
        self.synlist.append(h.Exp2Syn(self.soma(0.5))) # Inhibitory     
        self.synlist[-1].e = -75
        self.synlist[-1].tau1 = 1.0
        self.synlist[-1].tau2 = 7.0
    

In [ ]:
cell1.soma.psection()